In [ ]:
!pip install pandas
import pandas as pd
import numpy as np
import re
from typing import Dict
from pathlib import Path
import io
import warnings

Wczytanie danych

In [ ]:
# Sprawdzam czy sciezka data istnieje
data_dir = Path('./data')
if not data_dir.exists():
    raise FileNotFoundError(f"{data_dir} does not exist")

# Pobieram wszystkie pliki z rozszerzeniem CSV
csv_paths = sorted(data_dir.glob('*.csv'))

# Wczytuje pliki CSV z różnymi kodowaniami i separatorami
def try_read_csv(path):
    encodings = ['utf-8', 'utf-8-sig', 'latin-1', 'cp1250', 'cp1252']
    seps = [';', ',', '\t', '|']
    last_exc = None

    # Dla znanych separatorów preferuj engine='c' z low_memory=False (szybszy),
    # a jeśli się nie uda — engine='python' (bez low_memory).
    for enc in encodings:
        for sep in seps:
            try:
                df = pd.read_csv(path, encoding=enc, sep=sep, engine='c', low_memory=False, on_bad_lines='skip')
                return df, f"{enc}, sep='{sep}', engine=c"
            except Exception as e_c:
                last_exc = e_c
                try:
                    df = pd.read_csv(path, encoding=enc, sep=sep, engine='python', on_bad_lines='skip')
                    return df, f"{enc}, sep='{sep}', engine=python"
                except Exception as e_py:
                    last_exc = e_py

    # Sprawdzam encodings (typy zakodowań tekstu)(dodane z powodu warningów)
    for enc in encodings:
        try:
            df = pd.read_csv(path, encoding=enc, sep=None, engine='python', on_bad_lines='skip')
            return df, f"{enc}, sep=auto, engine=python"
        except Exception as e:
            last_exc = e

    # Warningi - niektóry tekst się nie wgrywał - dekodujemy ręcznie i czytamy jako utf-8 
    try:
        raw = path.read_bytes()
        text = None
        for enc in encodings:
            try:
                text = raw.decode(enc)
                break
            except Exception:
                continue
        if text is None:
            text = raw.decode('utf-8', errors='replace')
        df = pd.read_csv(io.StringIO(text), sep=None, engine='python', on_bad_lines='skip')
        return df, "decoded with replace, sep=auto, engine=python"
    except Exception as e:
        raise last_exc or e

In [ ]:
# Wczytuję wszystkie pliki CSV (wywołując funkcję powyżej)
# lista zawiera wszystkie df'y
loaded = []
for p in csv_paths:
    try:
        df, meta = try_read_csv(p)
    except Exception as e:
        print(f"Failed to read {p.name}: {e}")
    else:
        print(f"Read {p.name} ({meta}) — shape: {df.shape}")
        loaded.append((p.stem, df))

dataframes = [df for _, df in loaded]
dataframes_dict_dirty = dict(loaded)
print(f"Loaded {len(dataframes)} CSV files: {[name for name, _ in loaded]}")

Czyszczenie danych

In [ ]:
# Wykrywanie kolumn datowych
def _looks_like_date_series(s: pd.Series, sample_size: int = 200, threshold: float = 0.5) -> bool:
    """
    Sprawdza czy seria (kolumna) wygląda jak data.
    Szuka wzorców takich jak: 2020-01-15, 01/12/2020, oraz nazw miesięcy
    """
    # Konwertuje do tekstu i usuwa puste wartości
    vals = s.dropna().astype(str)
    if vals.empty:
        return False
    
    # Bierze losową próbkę (max 200 wartości) do analizy
    vals = vals.sample(min(len(vals), sample_size), random_state=0)
    
    # Regex dla dat: szuka liczb rozdzielonych myślnikami/kreskami/spacjami
    # np. 2020-01-15 lub 15/12/2020
    date_regex = re.compile(r'^\s*\d{1,4}(?:[\-\/\.\s]\d{1,2}){1,2}\s*$')
    
    # Regex dla nazw miesięcy (polska + angielska)
    month_names = re.compile(
        r'\b(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec|'
        r'styc|lut|mar|kwi|maj|cze|lip|sie|wrz|paź|lis|gru)\b', 
        re.I
    )
    
    # Sprawdza czy wartości pasują do wzoru daty LUB zawierają nazwę miesiąca
    matches = vals.str.match(date_regex) | vals.str.contains(month_names)
    
    # Jeśli co najmniej 50% wartości to daty, zwróć True
    return matches.sum() >= max(1, int(threshold * len(vals)))

In [ ]:
# Czyszczenie dataframe'u
def clean_df(df: pd.DataFrame, name: str = "", remove_outliers: bool = True) -> tuple:
    """
    Czyści ramkę danych i (opcjonalnie) usuwa outliery.
    Zwraca: (df_po_czyszczeniu, df_przed_usunieciem_outlierow)
    (Blok usuwania outlierów został zakomentowany.)
    """
    # Robimy kopię, żeby nie modyfikować oryginalnego obiektu przekazanego do funkcji.
    df = df.copy()

    # Lista kolumn, które chcemy POMINAĆ przy wykrywaniu outlierów (np. ID, nazwy)
    skip_columns_for_outliers = {
        'p_rok_od', 'p_kierunek_id', 'p_poziom', 'p_forma', 'p_uczelnia_id',
        'p_nazwa_uczelni', 'p_jedn', 'p_nazwa_jedn', 'p_woj', 'p_profil',
        'p_dziedzina_new', 'p_uczelnia_skrot', 'p_poziom_tekst_pl',
        'p_nazwa_kierunku_pelna', 'p_kierunek_nazwa', 'p_spec_nazwa', 'u_uczelnia_id', "u_n", "u_n_wzus","u_n_pozazus", "u_proc_wzus","u_proc_pozazus","u_n_dosw_rekr", "p_n",
        "u_n_dosw_studia", "p_n_wzus",
    }

    # KROK 1: Normalizuje nazwy kolumn
    # Zamienia na małe litery, usuwa spacje, zastępuje je podkreśleniami
    df.columns = [str(c).strip().lower().replace(" ", "_") for c in df.columns]
    # Usuwamy ewentualne duplikaty nazw kolumn (może się zdarzyć przy złym imporcie).
    df = df.loc[:, ~df.columns.duplicated()]

    # KROK 2: Usuwa całkowicie puste kolumny
    # axis=1 oznacza kolumny, how="all" oznacza całkowicie puste
    df.dropna(axis=1, how="all", inplace=True)

    # KROK 3: Naprawia tekstowe kolumny
    # Bierze tylko kolumny typu tekstowego (object)
    obj_cols = df.select_dtypes(include="object").columns.tolist()
    for c in obj_cols:
        # Konwertuj do string type, usuń spacje na początku/końcu, usuń znaki BOM
        df[c] = df[c].astype("string").str.strip().str.replace("\ufeff", "", regex=False)

         # Zamienia typowe znaczniki braku danych na pd.NA (None)
    df.replace({"": pd.NA, "NA": pd.NA, "N/A": pd.NA, "na": pd.NA, "-": pd.NA, "—": pd.NA, "None": pd.NA}, inplace=True)

    # KROK 4: Konwertuje kolumny liczbowe (tekst → liczby)
    for c in df.columns:
        # Jeśli kolumna jest tekstem, spróbuj skonwertować na liczby
        if df[c].dtype == "object" or pd.api.types.is_string_dtype(df[c]):
            # Usuń spacje wewnątrz liczb
            s = df[c].astype("string").str.replace(r"\s+", "", regex=True)
            # Zamień przecinki na kropki (polski format → międzynarodowy)
            s = s.str.replace(",", ".", regex=False)
            # Usuń wszystkie znaki oprócz cyfr, kropki i minusa
            s_clean = s.str.replace(r"[^0-9\.\-]", "", regex=True)
            # Konwertuj na liczby (błędy zamień na NaN)
            coerced = pd.to_numeric(s_clean, errors="coerce")
            non_null_count = coerced.notna().sum()
            # Jeśli co najmniej 30% wartości to liczby (nie puste), zaakceptuj konwersję
            if non_null_count > 0 and non_null_count >= max(1, int(0.3 * len(coerced))):
                df[c] = coerced

    # KROK 5: Konwertuje kolumny datowe (tekst → daty)
    for c in df.columns:
        # Jeśli kolumna jest tekstem
        if pd.api.types.is_object_dtype(df[c]) or pd.api.types.is_string_dtype(df[c]):
            # Sprawdź czy wygląda jak data
            if _looks_like_date_series(df[c]):
                # Wyłącz warningi podczas konwersji
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore", UserWarning)
                    # Konwertuj na datetime, przyjmując format DD/MM/YYYY
                    parsed = pd.to_datetime(df[c], errors="coerce", dayfirst=True)

                    # Jeśli co najmniej 30% wartości zostało skonwertowane, zaakceptuj
                if parsed.notna().sum() >= max(1, int(0.3 * len(parsed))):
                    df[c] = parsed

    # KROK 6: Usuwa puste wiersze i duplikaty
    # Usuwa wiersze które są całkowicie puste
    df.dropna(axis=0, how="all", inplace=True)
    # Usuwa wiersze które są całkowicie identyczne
    df.drop_duplicates(inplace=True)

    # Zapisujemy snapshot ramki po standardowym czyszczeniu, przed ewentualnym usuwaniem outlierów.
    df_before_outlier_removal = df.copy()

    # KROK 7: Usuwa wartości skrajne (outliers) — metoda IQR
    # POMIJAMY WYBRANE KOLUMNY (takie jak ID, kody, poziomy, kierunki itd.)
    """
    if remove_outliers:
        OUTLIER_QUANTILE = 0.99
        numeric_cols = [c for c in df.select_dtypes(include=[np.number]).columns.tolist()]

        mask_total = pd.Series(False, index=df.index)
        removed_entries = []

        for c in numeric_cols:
            if c in skip_columns_for_outliers:
                continue
            upper = df[c].quantile(OUTLIER_QUANTILE)
            if pd.isna(upper):
                continue
            mask = df[c].notna() & (df[c] > upper)
            if mask.any():
                for idx in df.index[mask]:
                    removed_entries.append((idx, c, df.at[idx, c]))
                mask_total |= mask

        if mask_total.any():
            total_to_remove = int(mask_total.sum())
            print(f"[outliers - percentyl {OUTLIER_QUANTILE}] {name} :: usuwam {total_to_remove} wierszy zawierających TOP {int((1-OUTLIER_QUANTILE)*100)}% wartości (limit 100 wypisanych):")
            shown = 0
            for idx, col, val in removed_entries:
                print(f"  index={idx}  column='{col}'  value={val}")
                shown += 1
                if shown >= 100:
                    remaining = len(removed_entries) - shown
                    if remaining > 0:
                        print(f"  ... oraz {remaining} pozostałych pozycji")
                    break
            df = df.loc[~mask_total].copy()
            print(f"Usunięto {total_to_remove} wierszy.")
    """

    # 8. Reset indeksu — porządkujemy indeksy po ewentualnych usunięciach.
    df.reset_index(drop=True, inplace=True)

    # Zwracamy: (wersja po czyszczeniu, snapshot przed outlierami)
    return df, df_before_outlier_removal

In [ ]:
# 1. DEFINICJA KOLUMN (Oszczędzamy RAM, wczytujemy tylko to co niezbędne)
# Zawsze korzystamy ze słownika zmiennych!
cols_to_load = [
    'P_DZIEDZINA', 
    'P_WWZ_P1', 'P_WWZ_P2', 'P_WWZ_P3', 'P_WWZ_P4', 'P_WWZ_P5',
    'P_WWB_P1', 'P_WWB_P2', 'P_WWB_P3', 'P_WWB_P4', 'P_WWB_P5'
]

# Wczytanie danych (zakładam, że plik z danymi to 'graduates-major-data.csv')
# Pamiętaj o odpowiednim kodowaniu i separatorze (często w ELA to ';')
df = pd.read_csv('graduates-major-data.csv', usecols=cols_to_load, sep=',') 

# 2. OBLICZENIE INDEKSU SYNTETYCZNEGO (Dla każdego roku osobno)
# Logika: Wynik = Zarobki (im więcej tym lepiej) - Ryzyko Bezrobocia (im mniej tym lepiej)
for i in range(1, 6):
    df[f'Score_P{i}'] = df[f'P_WWZ_P{i}'] - df[f'P_WWB_P{i}']

# 3. AGREGACJA (Grupowanie po Dziedzinie)
# Liczymy średni wynik dla całej dziedziny (np. Nauki Techniczne vs Humanistyczne)
df_grouped = df.groupby('P_DZIEDZINA')[[f'Score_P{i}' for i in range(1, 6)]].mean().reset_index()

# 4. TRANSFORMACJA WIDE TO LONG (Kluczowe dla wykresów liniowych!)
# Zmieniamy tabelę z szerokiej (kolumny lat obok siebie) na długą (jeden wiersz = jedna obserwacja rok-dziedzina)
df_melted = df_grouped.melt(
    id_vars=['P_DZIEDZINA'], 
    var_name='Okres', 
    value_name='Syntetyczny_Wskaznik'
)

# Czyszczenie kolumny 'Okres', żeby mieć liczby 1-5 zamiast stringów 'Score_P1'
df_melted['Rok_po_dyplomie'] = df_melted['Okres'].str.extract('(\d)').astype(int)

# 5. WIZUALIZACJA (Storytelling)
plt.figure(figsize=(12, 7))
sns.set_style("whitegrid")

sns.lineplot(
    data=df_melted, 
    x='Rok_po_dyplomie', 
    y='Syntetyczny_Wskaznik', 
    hue='P_DZIEDZINA',    # Każda dziedzina inny kolor
    marker='o',           # Kropki na punktach
    linewidth=2.5
)

plt.title('Syntetyczny Wskaźnik Sukcesu Zawodowego (WWZ - WWB) w czasie', fontsize=16, fontweight='bold')
plt.xlabel('Lata po uzyskaniu dyplomu', fontsize=12)
plt.ylabel('Wynik (Wyższy = Lepszy)', fontsize=12)
plt.axhline(0, color='black', linestyle='--', alpha=0.5) # Linia odniesienia (Zero)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', title='Dziedzina')
plt.tight_layout()
plt.show()